# Langchain Walkthrough

Inspired by: https://langchain-ai.github.io/langgraph/agents/agents/

Setup the environment

In [1]:
import dotenv
import os
import logfire
import json

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")


# configure logfire
logfire.configure(token=logfire_token)

# 2. Instrumenteer LangChain (automatisch loggen van chains, LLM calls, tools, etc.)
logfire.instrument()

<function logfire._internal.instrument.instrument.<locals>.decorator(func: 'Callable[P, R]') -> 'Callable[P, R]'>

1 Definieer een basis agent

In [2]:
from langgraph.prebuilt import create_react_agent

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")

basic_agent = create_react_agent(
    model=model,
    tools=[],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
)

Logfire project URL: ]8;id=443770;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\

Stel een vraag aan de agent

In [12]:
# Run the agent
result = basic_agent.invoke(
    {"messages": [{"role": "user", "content": "geef een defintie van een agent"}]}
)

print(result["messages"][-1].content)

12:49:44.235 LangGraph
12:49:44.247   agent
12:49:44.248     call_model
12:49:44.251       RunnableSequence
12:49:44.252         Prompt
12:49:44.252         ChatAnthropic
Een agent is een entiteit (mens, systeem of programma) die:

1. Waarnemingen verzamelt uit zijn omgeving via sensoren

2. Doelen of taken heeft die hij probeert te bereiken

3. Beslissingen kan nemen en acties kan uitvoeren om die doelen te bereiken

4. Beschikt over een vorm van intelligentie of logica om keuzes te maken

5. Kan leren en zich kan aanpassen op basis van eerdere ervaringen

In de kunstmatige intelligentie wordt een agent gezien als een computerprogramma dat:
- Zelfstandig kan waarnemen
- Kan redeneren
- Kan handelen
- Autonoom beslissingen kan nemen

Voorbeelden zijn:
- Software agents (chatbots, recommender systemen)
- Fysieke robots
- Virtuele assistenten
- Computerspel AI
- Geautomatiseerde handelssystemen

De kern is dat een agent gericht is op het bereiken van specifieke doelen in een bepaalde omg

Definieer tools voor de agent

In [13]:
tool_agent = create_react_agent(
    model=model,
    tools=[tf.calculate_btw, tf.calculate_discount, tf.get_inwoners_gemeente],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
)

config = {"configurable": {"thread_id": "1"}}

Test een aantal tool calls

In [5]:
result2 = tool_agent.invoke(
    {
        "messages": {
            "role": "user",
            "content": "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld",
        }
    },
    config=config,
)
print(result2["messages"][-1].content)

12:49:50.853 LangGraph
12:49:50.854   agent
12:49:50.854     call_model
12:49:50.855       RunnableSequence
12:49:50.855         Prompt
12:49:50.855         ChatAnthropic


Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:49:54.213     should_continue
12:49:54.214   tools
12:49:54.214 calculate_btw
             LangGraph
12:49:54.215   agent
12:49:54.215     call_model
12:49:54.215       RunnableSequence
12:49:54.215         Prompt
12:49:54.215         ChatAnthropic


Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:49:56.095     should_continue
12:49:56.096   tools
12:49:56.096 calculate_discount
             LangGraph
12:49:56.097   agent
12:49:56.097     call_model
12:49:56.097       RunnableSequence
12:49:56.097         Prompt
12:49:56.097         ChatAnthropic
12:50:00.299     should_continue
12:50:00.308 LangGraph
12:50:00.308   agent
12:50:00.308     call_model
12:50:00.308       RunnableSequence
12:50:00.308         Prompt
12:50:00.309         ChatAnthropic


Laat me je nu een duidelijk overzicht geven:

Oorspronkelijk bedrag (exclusief BTW): €100,00
BTW (21%): €21,00
Bedrag inclusief BTW: €121,00

Korting (15% over inclusief BTW): €18,15
Eindprijs na korting: €102,85

Samenvatting:
- Je koopt een product voor €100,00 exclusief BTW
- De BTW bedraagt €21,00
- Het bedrag inclusief BTW is €121,00
- Je krijgt 15% korting, wat neerkomt op €18,15
- Je uiteindelijke eindprijs is €102,85

Is er nog iets anders wat ik voor je kan verduidelijken?


In [6]:
result = tool_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien.",
            }
        ]
    }
)

print(result["messages"][-1].content)

Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:50:03.459     should_continue
12:50:03.461   tools
12:50:03.463     get_inwoners_gemeente
12:50:03.464   tools
12:50:03.466     get_inwoners_gemeente


Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:50:04.663   agent
12:50:04.663     call_model
12:50:04.663       RunnableSequence
12:50:04.663         Prompt
12:50:04.664         ChatAnthropic
12:50:10.538     should_continue
12:50:10.544 LangGraph
12:50:10.544   agent
12:50:10.544     call_model
12:50:10.545       RunnableSequence
12:50:10.545         Prompt
12:50:10.545         ChatAnthropic


Op basis van de gegevens voor het jaar 2024 kan ik u het volgende overzicht geven:

Top 7 gemeenten in Nederland naar aantal inwoners:
1. Amsterdam: 931.298 inwoners
2. Rotterdam: 670.610 inwoners
3. 's-Gravenhage (Den Haag): 566.221 inwoners
4. Utrecht: 374.238 inwoners
5. Eindhoven: 246.417 inwoners
6. Groningen: 243.768 inwoners
7. Tilburg: 229.836 inwoners

Extra informatie over Groningen:
- Aantal inwoners in 2024: 243.768
- Gemeente Groningen staat op de 6e plaats in de ranglijst van grootste gemeenten in Nederland

Enkele interessante observaties:
- Amsterdam heeft bijna 1 miljoen inwoners en is daarmee veruit de grootste gemeente
- De top 3 gemeenten (Amsterdam, Rotterdam, Den Haag) zijn duidelijk groter dan de rest
- De stad Groningen is een middelgrote gemeente met een substantieel aantal inwoners


In [7]:
result = tool_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan.",
            }
        ]
    }
)
print(result["messages"][-1].content)

Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:50:14.120     should_continue
12:50:14.121   tools
12:50:14.121     get_inwoners_gemeente


Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:50:15.167   agent
12:50:15.167     call_model
12:50:15.168       RunnableSequence
12:50:15.168         Prompt
12:50:15.168         ChatAnthropic
12:50:19.403     should_continue
12:50:19.421 LangGraph
12:50:19.422   agent
12:50:19.423     call_model
12:50:19.423       RunnableSequence
12:50:19.423         Prompt
12:50:19.424         ChatAnthropic


Nu kan ik het totale bedrag berekenen:
249.054 inwoners × 15 euro = 3.735.810 euro

Als alle inwoners van Eindhoven in 2025 15 euro zouden betalen, zou je een totaalbedrag van 3.735.810 euro krijgen. 

Enkele kanttekeningen:
1. Dit is een theoretische berekening
2. Het werkelijke aantal inwoners kan enigszins fluctueren
3. In de praktijk is het niet realistisch dat álle inwoners zouden betalen

Is er een specifieke reden waarom je dit wilt weten?


Voeg geheugen toe aan de agent

In [8]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent

checkpoint = InMemorySaver()

mem_agent = create_react_agent(
    model=model,
    tools=[tf.calculate_btw, tf.calculate_discount, tf.get_inwoners_gemeente],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
    checkpointer=checkpoint,
)

config = {"configurable": {"thread_id": "1"}}

result = mem_agent.invoke(
    {
        "messages": {
            "role": "user",
            "content": "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld",
        }
    },
    config=config,
)
print(result["messages"][-1].content)

Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:50:23.227     should_continue
12:50:23.228   tools
12:50:23.228 calculate_btw
             LangGraph
12:50:23.229   agent
12:50:23.229     call_model
12:50:23.229       RunnableSequence
12:50:23.229         Prompt
12:50:23.229         ChatAnthropic


Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence
Attribute 'langsmith.metadata.langgraph_path' mixes types str and int in attribute value sequence


12:50:26.044     should_continue
12:50:26.045   tools
12:50:26.045 calculate_discount
             LangGraph
12:50:26.046   agent
12:50:26.046     call_model
12:50:26.046       RunnableSequence
12:50:26.047         Prompt
12:50:26.047         ChatAnthropic
12:50:30.201     should_continue
12:50:30.213 LangGraph
12:50:30.216   agent
12:50:30.217     call_model
12:50:30.217       RunnableSequence
12:50:30.217         Prompt
12:50:30.218         ChatAnthropic
12:50:34.707     should_continue


Laat me dit voor je samenvatten:

Oorspronkelijke prijs (exclusief BTW): € 100,00
BTW (21%): € 21,00
Prijs inclusief BTW: € 121,00

Korting (15% over inclusief BTW): € 18,15
Eindprijs na korting: € 102,85

Dus je betaalt uiteindelijk € 102,85 voor product X, wat neerkomt op een effectieve korting van ongeveer 15% op de totaalprijs.


In [9]:
result = mem_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
            }
        ]
    },
    config=config,
)
print(result["messages"][-1].content)

Laat ik de gegevens voor je opzoeken uit de vorige berekening:

BTW bedrag voor product X: € 21,00
Korting bedrag: € 18,15

Deze bedragen heb ik zojuist voor je berekend in de vorige interactie, toen we de BTW en korting hebben uitgerekend voor het product dat oorspronkelijk 100 euro exclusief BTW kostte.
